Corpus (be.txt) taken from CC100 dataset: https://metatext.io/datasets/cc100-belarusian

In [6]:
import re
import gensim
from gensim.parsing.preprocessing import strip_punctuation, strip_short, strip_numeric, strip_multiple_whitespaces, remove_stopwords
from gensim.models.word2vec import LineSentence
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec

In [7]:
import logging
logging.basicConfig(level=logging.INFO, force = True)
logger = logging.getLogger()
logger.info("Logging initialized")

INFO:root:Logging initialized


In [8]:
def strip_trailing_newline(iterable):
    for i in iterable:
        yield i.rstrip()

CHARACTERS_MAP = {'ý': 'ў', 'i': 'і', 'ньн': 'нн', 'цьц': 'цц', 'сьц': 'сц', 'сьл':'сл'}
def replace_characters(iterable):
    for s in iterable:
        for k, v in CHARACTERS_MAP.items():
            s = s.replace(k, v)
        yield s

def split_sentences(iterable):
    for i in iterable:
        merged_dots = re.sub("[\.]+", ".", i)
        sentences = merged_dots.split('.')
        for s in sentences:
            yield s

STOPWORDS = ['на', 'не', 'што', 'да', 'па', 'як', 'за', 'для', 'гэта', 'ад', 'але', 'калі', 'пра', 'у', 'яго', 'якія', 'ён', 'іх', 'мы', 'каб', 'толькі', 'аб', 'таксама', 'які', 'ці', 'быў', 'было', 'яны', 'так', 'вы', 'яе', 'будзе', 'пры', 'яшчэ', 'тым', 'таму', 'вельмі', 'былі', 'можна', 'яна', 'пасля', 'пад', 'можа', 'дзе', 'якая', 'тут', 'была', 'трэба', 'тое', 'таго', 'або', 'гэтым', 'бо', 'ўсё', 'хто', 'ня', 'нас', 'гэтага', 'быць', 'гэты', 'ёсць', 'праз', 'ўжо', 'са', 'нават', 'то', 'мяне', 'ім','ва', 'той', 'усе', 'без', 'чым', 'мне', 'мае', 'сябе', 'гэтай', 'там', 'усё', 'вось', 'ды', 'каля', 'якіх', 'ты', 'якой', 'ўсе', 'жа', 'ужо', 'паводле', 'будуць', 'аднак', 'могуць', 'сваю', 'ні', 'сваёй', 'яму', 'свае', 'гэтыя', 'проста', 'ўсіх', 'якім', 'падчас', 'тады', 'свой', 'вас', 'паміж', 'нам', 'раз', 'сваіх', 'нашай', 'менавіта', 'перад', 'вам', 'тых','зь','такім', 'свайго', 'над', 'зараз', 'амаль', 'чаму', 'ёй', 'чынам', 'напрыклад', 'якога', 'якое', 'сваім', 'можаце', 'га', 'хоць', 'бы', 'тыя', 'такія', 'потым', 'адным', 'такі', 'якую', 'сабе','сам','гэтых','мая','наш','зусім','чаго','наша','зноў','дык','такіх','нашага','адразу','каго','самі','ст','ну','нашы','нашым','самы','яно','гэтае','дзеля','куды','by','гг']
WORD_MAP = {}
with open('word-map', 'r') as map_file:
    for m in map_file:
        v = m.rstrip().split(':')
        WORD_MAP[v[0]] = v[1]
def preprocess_sentences(iterable):
    for i in iterable:
        s = strip_multiple_whitespaces(strip_numeric(strip_short(strip_punctuation(i))))
        s = s.lower()
        s = re.sub("[«»“”„…—°′²]", "", s)
        s = remove_stopwords(s, stopwords=STOPWORDS)
        s = ' '.join([WORD_MAP.get(w, w) for w in s.split()])
        yield s

def remove_short_lines(iterable):
    for i in iterable:
        if not i.isspace() and len(i) >= 20:
            yield i


In [9]:
with open('be.txt', 'r') as original_file:
    with open('sentences.txt', 'w') as sentences_file:
        for s in remove_short_lines(preprocess_sentences(split_sentences(replace_characters(strip_trailing_newline(original_file))))):
            sentences_file.write(s + "\n")

In [10]:
loss_list = []
class Callback(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        loss_list.append(loss)
        print('Loss after epoch {}:{}'.format(self.epoch, loss))
        model.running_training_loss = 0.0
        self.epoch = self.epoch + 1

In [11]:
model = Word2Vec(vector_size=100, window=3, min_count=10, workers=5)

INFO:gensim.utils:Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=100, alpha=0.025>', 'datetime': '2023-04-16T20:08:38.535456', 'gensim': '4.3.1', 'python': '3.11.3 | packaged by conda-forge | (main, Apr  6 2023, 09:05:00) [Clang 14.0.6 ]', 'platform': 'macOS-12.6-x86_64-i386-64bit', 'event': 'created'}


In [12]:
sentences = LineSentence('sentences.txt')

In [13]:
model.build_vocab(sentences, progress_per=5000000)

INFO:gensim.models.word2vec:collecting all words and their counts
INFO:gensim.models.word2vec:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #5000000, processed 44450539 words, keeping 1130486 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #10000000, processed 88988536 words, keeping 1640652 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #15000000, processed 133392566 words, keeping 2022817 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #20000000, processed 177908716 words, keeping 2345463 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #25000000, processed 222304163 words, keeping 2636351 word types
INFO:gensim.models.word2vec:collected 2666025 word types from a corpus of 227126066 raw words and 25549177 sentences
INFO:gensim.models.word2vec:Creating a fresh vocabulary
INFO:gensim.utils:Word2Vec lifecycle event {'msg': 'effective_min_count=10 retains 481937 uniqu

In [14]:
model.train(sentences, epochs=50, total_examples=model.corpus_count, total_words=model.corpus_total_words, compute_loss=True, report_delay=30, callbacks=[Callback()])


INFO:gensim.utils:Word2Vec lifecycle event {'msg': 'training model with 5 workers on 481937 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=3 shrink_windows=True', 'datetime': '2023-04-16T20:10:18.906517', 'gensim': '4.3.1', 'python': '3.11.3 | packaged by conda-forge | (main, Apr  6 2023, 09:05:00) [Clang 14.0.6 ]', 'platform': 'macOS-12.6-x86_64-i386-64bit', 'event': 'train'}
INFO:gensim.models.word2vec:EPOCH 0 - PROGRESS: at 0.45% examples, 994905 words/s, in_qsize 9, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 0 - PROGRESS: at 16.22% examples, 1152278 words/s, in_qsize 8, out_qsize 1
INFO:gensim.models.word2vec:EPOCH 0 - PROGRESS: at 31.82% examples, 1149678 words/s, in_qsize 10, out_qsize 2
INFO:gensim.models.word2vec:EPOCH 0 - PROGRESS: at 47.50% examples, 1151328 words/s, in_qsize 9, out_qsize 1
INFO:gensim.models.word2vec:EPOCH 0 - PROGRESS: at 63.16% examples, 1150641 words/s, in_qsize 9, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 0 - PROGRESS:

Loss after epoch 0:42427652.0


INFO:gensim.models.word2vec:EPOCH 1 - PROGRESS: at 0.51% examples, 1103722 words/s, in_qsize 10, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 1 - PROGRESS: at 16.23% examples, 1153016 words/s, in_qsize 9, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 1 - PROGRESS: at 32.12% examples, 1160461 words/s, in_qsize 9, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 1 - PROGRESS: at 48.06% examples, 1164972 words/s, in_qsize 8, out_qsize 1
INFO:gensim.models.word2vec:EPOCH 1 - PROGRESS: at 63.88% examples, 1163931 words/s, in_qsize 10, out_qsize 1
INFO:gensim.models.word2vec:EPOCH 1 - PROGRESS: at 78.72% examples, 1149253 words/s, in_qsize 10, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 1 - PROGRESS: at 92.91% examples, 1131302 words/s, in_qsize 9, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 1: training on 227126066 raw words (220366760 effective words) took 195.8s, 1125591 effective words/s


Loss after epoch 1:39036736.0


INFO:gensim.models.word2vec:EPOCH 2 - PROGRESS: at 0.42% examples, 916347 words/s, in_qsize 9, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 2 - PROGRESS: at 12.81% examples, 910605 words/s, in_qsize 9, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 2 - PROGRESS: at 28.58% examples, 1032973 words/s, in_qsize 9, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 2 - PROGRESS: at 44.52% examples, 1078896 words/s, in_qsize 10, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 2 - PROGRESS: at 59.85% examples, 1090263 words/s, in_qsize 10, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 2 - PROGRESS: at 75.32% examples, 1099717 words/s, in_qsize 9, out_qsize 1
INFO:gensim.models.word2vec:EPOCH 2 - PROGRESS: at 90.97% examples, 1107988 words/s, in_qsize 9, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 2: training on 227126066 raw words (220366843 effective words) took 198.6s, 1109519 effective words/s


...

INFO:gensim.models.word2vec:EPOCH 48 - PROGRESS: at 0.48% examples, 1051574 words/s, in_qsize 10, out_qsize 1
INFO:gensim.models.word2vec:EPOCH 48 - PROGRESS: at 15.54% examples, 1104616 words/s, in_qsize 8, out_qsize 1
INFO:gensim.models.word2vec:EPOCH 48 - PROGRESS: at 30.54% examples, 1104020 words/s, in_qsize 0, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 48 - PROGRESS: at 45.48% examples, 1102191 words/s, in_qsize 9, out_qsize 1
INFO:gensim.models.word2vec:EPOCH 48 - PROGRESS: at 60.12% examples, 1095148 words/s, in_qsize 10, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 48 - PROGRESS: at 75.16% examples, 1097398 words/s, in_qsize 9, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 48 - PROGRESS: at 90.39% examples, 1100864 words/s, in_qsize 7, out_qsize 2
INFO:gensim.models.word2vec:EPOCH 48: training on 227126066 raw words (220365820 effective words) took 200.3s, 1100191 effective words/s


Loss after epoch 48:33460040.0


INFO:gensim.models.word2vec:EPOCH 49 - PROGRESS: at 0.43% examples, 939031 words/s, in_qsize 9, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 49 - PROGRESS: at 15.73% examples, 1117744 words/s, in_qsize 9, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 49 - PROGRESS: at 31.06% examples, 1122381 words/s, in_qsize 9, out_qsize 1
INFO:gensim.models.word2vec:EPOCH 49 - PROGRESS: at 46.26% examples, 1120773 words/s, in_qsize 9, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 49 - PROGRESS: at 61.56% examples, 1121273 words/s, in_qsize 9, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 49 - PROGRESS: at 77.35% examples, 1129445 words/s, in_qsize 10, out_qsize 1
INFO:gensim.models.word2vec:EPOCH 49 - PROGRESS: at 93.17% examples, 1134726 words/s, in_qsize 9, out_qsize 0
INFO:gensim.models.word2vec:EPOCH 49: training on 227126066 raw words (220368870 effective words) took 193.9s, 1136686 effective words/s
INFO:gensim.utils:Word2Vec lifecycle event {'msg': 'training on 11356303300 raw words (1101834

Loss after epoch 49:33048170.0


(11018347207, 11356303300)

In [15]:
model.save("word2vec-100-bel-cc100.model")

INFO:gensim.utils:Word2Vec lifecycle event {'fname_or_handle': 'word2vec-100-bel-cc100.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2023-04-16T22:51:28.103868', 'gensim': '4.3.1', 'python': '3.11.3 | packaged by conda-forge | (main, Apr  6 2023, 09:05:00) [Clang 14.0.6 ]', 'platform': 'macOS-12.6-x86_64-i386-64bit', 'event': 'saving'}
INFO:gensim.utils:storing np array 'vectors' to word2vec-100-bel-cc100.model.wv.vectors.npy
INFO:gensim.utils:storing np array 'syn1neg' to word2vec-100-bel-cc100.model.syn1neg.npy
INFO:gensim.utils:not storing attribute cum_table
INFO:gensim.utils:saved word2vec-100-bel-cc100.model


In [100]:
model.wv.most_similar('гогат', topn=20)

[('рогат', 0.7393548488616943),
 ('віскат', 0.7063933610916138),
 ('шэпт', 0.7021474242210388),
 ('гул', 0.6958292722702026),
 ('прарэзлівы', 0.6882970333099365),
 ('хрып', 0.681495726108551),
 ('гучнейшы', 0.6771256327629089),
 ('звонкі', 0.6758430600166321),
 ('пошум', 0.674797534942627),
 ('трэск', 0.6702352166175842),
 ('шум', 0.6626041531562805),
 ('свіст', 0.6624439358711243),
 ('тупат', 0.6604283452033997),
 ('прыцішаны', 0.659925639629364),
 ('выкрык', 0.6570577025413513),
 ('прыглушаны', 0.653853178024292),
 ('рокат', 0.6535115838050842),
 ('віск', 0.6509947180747986),
 ('хрыплы', 0.649290919303894),
 ('грукат', 0.6481378674507141)]

In [28]:
model.wv.index_to_key

['беларускі',
 'беларусь',
 'год',
 'час',
 'больш',
 'дзень',
 'людзі',
 'мова',
 'чалавек',
 'жыццё',
 'праца',
 'іншы',
 'годзе',
 'адзін',
 'краіна',
 'новы',
 'рэспубліка',
 'вялікі',
 'гісторыя',
 'месца',
 'мінск',
 'гульня',
 'горад',
 'два',
 'культура',
 'сайт',
 'раён',
 'старонка',
 'правіць',
 'дзеці',
 'цяпер',
 'розны',
 'шмат',
 'справа',
 'сёння',
 'першы',
 'права',
 'разам',
 'матэрыял',
 'пытанне',
 'няма',
 'кожны',
 'адукацыя',
 'імя',
 'нацыянальны',
 'свет',
 'дзяржаўны',
 'слова',
 'беларус',
 'вынік',
 'арганізацыя',
 'кніга',
 'пачатак',
 'дзейнасць',
 'літаратура',
 'некалькі',
 'вайна',
 'развіццё',
 'вобласць',
 'дом',
 'сярод',
 'інфармацыя',
 'стала',
 'дзяржава',
 'цэнтр',
 'вёска',
 'супраць',
 'школа',
 'праграма',
 'лік',
 'яўляецца',
 'сказаў',
 'улада',
 'зямля',
 'зрабіць',
 'радыё',
 'прадстаўнік',
 'старшыня',
 'відэа',
 'кампанія',
 'пакуль',
 'тэрыторыя',
 'спасылка',
 'заўсёды',
 'прэзідэнт',
 'умова',
 'сувязь',
 'апошні',
 'добра',
 'тры',


In [37]:
model.wv.get_vecattr("чарадзей", "count")

1379